In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [11]:
import os
os.environ['PYTHONHASHSEED']='0'

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,log_loss,roc_auc_score
from math import log10
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from scipy.stats import skew, kurtosis
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
data_path = '/home/kesci/input/smart_edu7557/'
feature_path = '/home/kesci/work/feature/'
out_path = '/home/kesci/work/out/'
all_knowledge = pd.read_csv(open(data_path + 'all_knowledge.csv',encoding='utf8'))
course1_exams = pd.read_csv(open(data_path + 'course1_exams.csv',encoding='utf8'))#85,369 考点依次向后，最后为综合性考试
course2_exams = pd.read_csv(open(data_path + 'course2_exams.csv',encoding='utf8')) #94,368
course3_exams = pd.read_csv(open(data_path + 'course3_exams.csv',encoding='utf8'))#70，263
course4_exams = pd.read_csv(open(data_path + 'course4_exams.csv',encoding='utf8'))#68,220
course5_exams = pd.read_csv(open(data_path + 'course5_exams.csv',encoding='utf8'))#90,336
course6_exams = pd.read_csv(open(data_path + 'course6_exams.csv',encoding='utf8'))#54,147
course7_exams = pd.read_csv(open(data_path + 'course7_exams.csv',encoding='utf8'))#78,317
course8_exams = pd.read_csv(open(data_path + 'course8_exams.csv',encoding='utf8'))#78,224
train = pd.read_csv(open(data_path + 'exam_score.csv',encoding='utf8')) #共8门课 65500,4
student = pd.read_csv(open(data_path + 'student.csv',encoding='utf8'))#500,2
course = pd.read_csv(open(data_path + 'course.csv',encoding='utf8'))#8,2
test = pd.read_csv(open(data_path + 'submission_s2.csv',encoding='utf8'))#8000,5
exams = pd.DataFrame({"exam_id":[],"level_1":[],0:[],"course":[]})

# 模型1：章节考试模型

In [14]:
for i in range(1,9):
    s=0
    exec("""s = course{}_exams.set_index("exam_id").stack().to_frame().reset_index()""".format(i))
    exam_list = s["exam_id"].drop_duplicates().values
    s_temp = pd.DataFrame({"exam_id":exam_list,"exam_time":range(len(exam_list))})
    s = s.merge(s_temp,how='left',on="exam_id")
    s["course"] = "course{}".format(i)
    exams = exams.append(s)

exams = exams.rename(columns={0:"score"})
# 合并所有和考试相关的信息
exams = exams.merge(all_knowledge,how='left',left_on=["course","level_1"],right_on=["course","knowledge_point"])
exams = exams.merge(course,how='left',on='course')
exams["hardvalue"] = exams["score"]*exams["complexity"]
exams["rank"] = exams.groupby("exam_id")["score"].rank(ascending=False,method='first')
exams["score"] = exams["score"].replace(0,np.nan)
#将所有考试信息归到一维
agg = {
    "score":["count"],#考了几个知识点
    "hardvalue":["sum","max"] #难度值、难度最高的知识点有多高
}
exams_group = exams.groupby(["exam_id","exam_time","course","course_class"],as_index=False).agg(agg) #617,2
exams_group.columns=["exam_id","exam_time","course","course_class","knowledge_cnt","hardvalue_sum","hardvalue_max"]
exams_group = exams_group.merge(exams.loc[exams["rank"]==1][["exam_id","section","category"]],how="left",on=["exam_id"])#本次考试的知识点所属领域
exams_group = exams_group.sort_values(by=["course_class","course","exam_time"]).reset_index(drop=True)

#学生维度
train["is_train"]=1
test["is_train"]=0
test.rename(columns = {'pred':'score'},inplace=True)
data_all = train.append(test)
data_all = data_all.merge(student,how='left',on='student_id')
data_all = data_all.merge(exams_group,how='left',on=["exam_id","course"])
data_all["effotvalue"] = data_all["score"]/data_all["hardvalue_sum"]
bin_labels = [0,1,2,3,4,5,6,7]
data_all["hardvalue_rank"]=pd.cut(data_all["hardvalue_sum"], 8, right=True, labels=bin_labels, retbins=False, precision=3, include_lowest=False)
#当前章节第几次考试,测试集中3/16次考试更换了知识点,category_rank为nan则说明为阶段性考试
data_tmp = data_all.sort_values(by=["student_id","course","section","knowledge_cnt"])
data_tmp = data_tmp.drop_duplicates(["student_id","course","section"])
data_tmp["category_rank"] = data_tmp.groupby(["student_id","course","category"])["exam_time"].rank(method="first")
data_all = data_all.merge(data_tmp[["student_id","course","exam_time","category_rank"]],how='left',on=["student_id","course","exam_time"])
#当前数据输出
# data_all.to_csv(feature_path + "data0809.csv", index=False)

#聚合
exam_his={"course1":18,"course2":21,"course3":14,"course4":14,
          "course5":22,"course6":10,"course7":16,"course8":16}
# exam_his={"course1":20,"course2":23,"course3":16,"course4":16,
#           "course5":24,"course6":12,"course7":18,"course8":18} 
          
data_all["exam_time_tmp"] = data_all["course"].replace(exam_his)
data_all["exam_time01"] =data_all["exam_time"]-(0.1*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time02"] =data_all["exam_time"]-(0.2*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time03"] =data_all["exam_time"]-(0.3*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time04"] =data_all["exam_time"]-(0.4*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time05"] =data_all["exam_time"]-(0.5*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time06"] =data_all["exam_time"]-(0.6*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time07"] =data_all["exam_time"]-(0.7*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time08"] =data_all["exam_time"]-(0.8*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time09"] =data_all["exam_time"]-(0.9*data_all["exam_time_tmp"]).astype(int)
data_all["exam_time10"] =data_all["exam_time"]-(data_all["exam_time_tmp"]).astype(int)

#对历史努力值进行统计
agg = {
    "score":["mean","median","std","max","min"],
    "category_his":["count"]
    # "score":["mean","median","std","max","min",'skew'],
    # "category_his":["count"],
    # "effotvalue":["mean","median","std","max","min",'skew']
}
for i in ["01","02","03","04","05","06","07","08","09","10"]:
    print(i)
    data_union = data_all[["student_id","course","exam_id","exam_time","exam_time{}".format(i),"category"]]\
        .merge(data_all[["student_id","course","exam_time","category", "effotvalue","score","hardvalue_sum"]].rename(columns={"exam_time":"exam_time_his","category":"category_his"}),
               how='left',on=["student_id","course"])
    data_union = data_union.loc[(data_union["exam_time"+i]<data_union["exam_time_his"])&(data_union["exam_time_his"]<data_union["exam_time"])]
    data_union["score"] = data_union["score"].replace(0,np.nan) #将缺考的努力值置为空
    data_tmp = data_union.groupby(["student_id","course","exam_id"],as_index=False).agg(agg)
    data_tmp.columns = [j[0]+"_"+j[1]+i for j in data_tmp.columns ]
    data_tmp.columns = ["student_id","course","exam_id"]+list(data_tmp.columns[3:])
    #计算应该对几条数据做聚合，筛选数据不足的行,如：第二次考试仅可以获取第一次考试的信息，信息量少于0.5/0.7/1，故舍弃
    data_tmp["cnt"] = (data_tmp["course"].replace(exam_his)*(int(i[0])+0.1*int(i[1]))).astype(int)-1
    data_tmp = data_tmp.loc[data_tmp["category_his_count"+i]==data_tmp["cnt"]]
    del data_tmp["cnt"]
    del data_tmp["category_his_count"+i]
    data_all = data_all.merge(data_tmp, how='left', on=["student_id", "course", "exam_id"])
    if i=="10":
        #筛选近期考点属于同一大类的考试进行统计
        data_tmp2 = data_union.loc[data_union["category"]==data_union["category_his"]]
        data_tmp2 = data_tmp2.groupby(["student_id", "course", "exam_id"], as_index=False)["score"].agg(["mean","median","std","max","min"])
        data_tmp2 = data_tmp2.reset_index()
        data_tmp2.columns = list(data_tmp2.columns[:3])+["his_category_"+j for j in data_tmp2.columns[3:]]
        data_all = data_all.merge(data_tmp2,how='left',on=["student_id", "course", "exam_id"])
        #近3次同类考试的分数和难度值
        data_tmp2 = data_union.loc[data_union["category"] == data_union["category_his"]]
        data_tmp2["his_rank"]=data_tmp2.groupby(["student_id", "course", "exam_id"])["exam_time_his"].rank(ascending=False,method='first')
        for k in [1,2,3]:
            data_tmp3 = data_tmp2.loc[data_tmp2.his_rank==k]
            data_tmp3 = data_tmp3[["student_id", "course", "exam_id","score","hardvalue_sum"]].rename(columns={"score":"score_last{}".format(k),"hardvalue_sum":"hardvalue_last{}".format(k)})
            data_all=data_all.merge(data_tmp3,how='left',on=["student_id", "course", "exam_id"])
del data_all["exam_time_tmp"],data_all["exam_time05"],data_all["exam_time07"],data_all["exam_time10"]
data_all.to_csv(feature_path+"data_plus_score0825.csv", index=False)


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


01
02
03
04
05
06
07
08
09
10


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
data_all = pd.read_csv(open(feature_path+"data_plus_score0825.csv",encoding="utf8"))
###################### 增加期中 期末 学期
is_midterm={"course1":[5,17,27,37,48,58,68,78],
    "course2":[5,17,28,39,51,63,75,87],
    "course3":[4,12,21,30,40,48,56,65],
    "course4":[4,13,22,30,38,47,54,63],
    "course5":[4,15,27,38,49,59,70,83],
    "course6":[3,11,17,23,30,37,43,50],
    "course7":[4,14,24,34,44,54,63,72],
    "course8":[4,14,24,35,45,54,63,73]
}
is_final={"course1":[-1,11,22,31,43,53,63,72,84],
    "course2":[-1,11,22,33,45,57,69,80,93],
    "course3":[-1,8,16,25,35,44,52,60,69],
    "course4":[-1,8,17,26,33,43,50,58,67],
    "course5":[-1,9,21,33,43,54,64,76,89],
    "course6":[-1,7,14,20,26,33,40,46,53],
    "course7":[-1,8,19,29,39,49,58,67,77],
    "course8":[-1,9,19,29,40,49,58,68,77]
}
data_all["is_midterm"] = 0 #是否期中
data_all["is_final"] = 0 #是否期末
data_all["term"] = 0 #学期数
for key in is_midterm.keys():
    data_all.loc[(data_all["course"]==key)&(data_all["exam_time"].isin(is_midterm[key])),"is_midterm"]=1
    data_all.loc[(data_all["course"] == key) & (data_all["exam_time"].isin(is_final[key])), "is_final"] = 1
    term = data_all.loc[data_all["course"]==key,"exam_time"]
    data_all.loc[data_all["course"] == key, "term"] = pd.cut(term,is_final[key],labels=False)
############################
print(data_all.shape)
data_all = data_all.loc[data_all.score!=0] #去除包含0的值
print(data_all.shape)
# data_all2 = pd.read_csv(open(feature_path+"gupengfeaturegupeng_course_feature.csv",encoding="utf8"))

# 方案1：去除缺失较多的行
# data_all = data_all.loc[np.isnan(data_all["category_rank"])==0] #去除阶段性考试
data_all = data_all.loc[np.isnan(data_all["his_category_mean"])==0] #去除his_category空值
data_all = data_all.loc[np.isnan(data_all["score_mean05"])==0] #去除05空值
data_all = data_all.loc[np.isnan(data_all["score_mean07"])==0] #去除07空值
data_all = data_all.loc[np.isnan(data_all["score_mean10"])==0] #去除10空值
print(data_all.shape)

data_all = data_all.reset_index(drop=True)
# data_all = data_all.merge(data_all2,on = ['student_id','course','exam_id','is_train'],how = 'left')
data_all_temp = data_all.copy()
le = LabelEncoder()
data_all_temp["le_course"] = le.fit_transform(data_all_temp["course"])
le = LabelEncoder()
data_all_temp["le_exam_id"] = le.fit_transform(data_all_temp["exam_id"])
le = LabelEncoder()
data_all_temp["le_course_class"] = le.fit_transform(data_all_temp["course_class"])
le = LabelEncoder()
data_all_temp["le_category"] = le.fit_transform(data_all_temp["category"])

y = "score"

features = ["gender","exam_time","knowledge_cnt","category_rank"]\
          +["hardvalue_sum","hardvalue_max","hardvalue_rank"] \
          + [i for i in data_all_temp.columns if "his_category_" in i] \
          +["le_course","le_course_class","le_category"] \
          + [i for i in data_all_temp.columns if "05" in i] \
          + [i for i in data_all_temp.columns if "07" in i] \
          + [i for i in data_all_temp.columns if "_last" in i] \
            + [i for i in data_all_temp.columns if "10" in i] \
            + [i for i in data_all_temp.columns if "01" in i] \
            + [i for i in data_all_temp.columns if "03" in i] \
            + ["term","is_midterm","is_final"]\
            # + [i for i in data_all_temp.columns if "02" in i] \
            # + [i for i in data_all_temp.columns if "04" in i] \

"""
4.732
02 4.729
"""

categorical_feats = ["le_course","le_course_class","le_category","gender"]
print(features)
data_all_temp.head()

(308500, 87)
(307899, 87)
(168014, 87)
['gender', 'exam_time', 'knowledge_cnt', 'category_rank', 'hardvalue_sum', 'hardvalue_max', 'hardvalue_rank', 'his_category_mean', 'his_category_median', 'his_category_std', 'his_category_max', 'his_category_min', 'le_course', 'le_course_class', 'le_category', 'score_mean05', 'score_median05', 'score_std05', 'score_max05', 'score_min05', 'score_mean07', 'score_median07', 'score_std07', 'score_max07', 'score_min07', 'score_last1', 'hardvalue_last1', 'score_last2', 'hardvalue_last2', 'score_last3', 'hardvalue_last3', 'score_mean10', 'score_median10', 'score_std10', 'score_max10', 'score_min10', 'exam_time01', 'score_mean01', 'score_median01', 'score_std01', 'score_max01', 'score_min01', 'exam_time03', 'score_mean03', 'score_median03', 'score_std03', 'score_max03', 'score_min03', 'term', 'is_midterm', 'is_final']


,student_id,course,exam_id,score,is_train,gender,exam_time,course_class,knowledge_cnt,hardvalue_sum,hardvalue_max,section,category,effotvalue,hardvalue_rank,category_rank,exam_time01,exam_time02,exam_time03,exam_time04,exam_time06,exam_time08,exam_time09,score_mean01,score_median01,score_std01,score_max01,score_min01,score_mean02,score_median02,score_std02,score_max02,score_min02,score_mean03,score_median03,score_std03,score_max03,score_min03,score_mean04,score_median04,score_std04,score_max04,score_min04,score_mean05,score_median05,score_std05,score_max05,score_min05,score_mean06,score_median06,score_std06,score_max06,score_min06,score_mean07,score_median07,score_std07,score_max07,score_min07,score_mean08,score_median08,score_std08,score_max08,score_min08,score_mean09,score_median09,score_std09,score_max09,score_min09,score_mean10,score_median10,score_std10,score_max10,score_min10,his_category_mean,his_category_median,his_category_std,his_category_max,his_category_min,score_last1,hardvalue_last1,score_last2,hardvalue_last2,score_last3,hardvalue_last3,is_midterm,is_final,term,le_course,le_exam_id,le_course_class,le_category
0,230748,course1,E6OBmZgw,74.0,1,0,17.0,course_class1,18,275.0,45.0,S:11,C:2,0.269091,5,NaN,16.0,14.0,12.0,10.0,7.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,74.0,74.0,5.656854,78.0,70.0,73.75,73.5,4.349329,78.0,70.0,74.333333,73.5,4.926121,81.0,70.0,73.375,72.0,4.897157,81.0,67.0,73.888889,74.0,4.833333,81.0,67.0,73.818182,74.0,5.211875,81.0,67.0,73.692308,74.0,4.922476,81.0,67.0,73.400000,73.0,4.656792,81.0,67.0,73.117647,71.0,4.428351,81.0,67.0,76.50,77.5,4.654747,81.0,70.0,78.0,248.0,70.0,342.0,77.0,208.0,1,0,1,0,92,0,11
1,186851,course1,E6OBmZgw,70.0,1,0,17.0,course_class1,18,275.0,45.0,S:11,C:2,0.254545,5,NaN,16.0,14.0,12.0,10.0,7.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,79.0,79.0,4.242641,82.0,76.0,76.00,75.0,4.320494,82.0,72.0,76.000000,75.5,3.405877,82.0,72.0,78.500,76.5,5.554921,88.0,72.0,80.111111,77.0,7.096556,93.0,72.0,82.090909,82.0,8.336121,98.0,72.0,81.307692,79.0,7.888648,98.0,72.0,80.400000,77.0,7.707140,98.0,72.0,79.294118,76.0,7.982518,98.0,67.0,74.75,75.0,2.217356,77.0,72.0,76.0,248.0,72.0,342.0,74.0,208.0,1,0,1,0,92,0,11
2,478370,course1,E6OBmZgw,78.0,1,0,17.0,course_class1,18,275.0,45.0,S:11,C:2,0.283636,5,NaN,16.0,14.0,12.0,10.0,7.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,80.0,80.0,1.414214,81.0,79.0,79.25,80.0,2.362908,81.0,76.0,79.666667,80.0,3.983298,86.0,75.0,78.125,77.5,4.486090,86.0,72.0,78.777778,79.0,4.630815,86.0,72.0,78.909091,79.0,4.826066,86.0,72.0,80.230769,81.0,5.464337,88.0,72.0,80.666667,81.0,5.354126,88.0,72.0,80.882353,81.0,5.194737,88.0,72.0,81.00,81.0,4.082483,86.0,76.0,81.0,248.0,76.0,342.0,81.0,208.0,1,0,1,0,92,0,11
3,692328,course1,E6OBmZgw,80.0,1,1,17.0,course_class1,18,275.0,45.0,S:11,C:2,0.290909,5,NaN,16.0,14.0,12.0,10.0,7.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,82.0,82.0,2.828427,84.0,80.0,80.25,81.0,3.862210,84.0,75.0,82.166667,82.5,4.622409,89.0,75.0,82.375,82.5,4.068608,89.0,75.0,83.444444,83.0,4.977728,92.0,75.0,84.090909,83.0,5.889899,95.0,75.0,83.923077,83.0,5.407734,95.0,75.0,83.733333,83.0,5.035115,95.0,75.0,83.235294,82.0,4.943802,95.0,75.0,81.50,81.0,5.802298,89.0,75.0,80.0,248.0,75.0,342.0,82.0,208.0,1,0,1,0,92,0,11
4,509128,course1,E6OBmZgw,72.0,1,0,17.0,course_class1,18,275.0,45.0,S:11,C:2,0.261818,5,NaN,16.0,14.0,12.0,10.0,7.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,76.5,76.5,6.363961,81.0,72.0,73.25,71.5,5.315073,81.0,69.0,74.333333,74.0,4.457204,81.0,69.0,75.625,76.5,4.470139,81.0,69.0,76.777778,77.0,5.426274,86.0,69.0,77.181818,77.0,6.096199,87.0,69.0,77.307692,77.0,5.588450,87.0,69.0,77.000000,77.0,5.250850,87.0,69.0,76.588235,76.0,5.075170,87.0,69.0,72.00,71.5,2.943920,76.0,69.0,72.0,248.0,69.0,342.0,71.0,208.0,1,0,1,0,92,0,11


In [16]:
from math import log10
# categorical_feats = []
train = data_all_temp[data_all.is_train==1]
test = data_all_temp[data_all.is_train==0]

folds = KFold(n_splits=5, shuffle=True, random_state=2333)

oof = np.zeros(len(train))
predictions = np.zeros(len(test))
train_x = train[features]
test_x = test[features]
train_y = train[y]

param = {'objective': 'regression',
         'num_leaves': 2**5, #2**5
         'min_data_in_leaf': 25,#
         'max_depth': 5,  #
         'learning_rate': 0.02, #0.02
         'lambda_l1': 0.13,#0.13
         "boosting": "gbdt",
         "feature_fraction": 0.7,#0.85
         'bagging_freq': 8,
         "bagging_fraction": 0.9, #0.9
         "metric": 'rmse',
         "verbosity": -1,
         "random_state": 2333,
         "num_threads" : 50}
# lgb
model = "lgb"
feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x.values, train_y.values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(train_x.iloc[trn_idx],
                           label=train_y.iloc[trn_idx],
                           categorical_feature=categorical_feats
                           )
    val_data = lgb.Dataset(train_x.iloc[val_idx],
                           label=train_y.iloc[val_idx],
                           categorical_feature=categorical_feats
                           )

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=500,early_stopping_rounds=100)
    #n*6矩阵
    oof[val_idx] = clf.predict(train_x.iloc[val_idx], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions += clf.predict(test_x, num_iteration=clf.best_iteration) / folds.n_splits
feature_importance = feature_importance_df[["Feature", "importance"]].groupby("Feature").mean().sort_values(by="importance",
ascending=False)

from sklearn import metrics
print(round(metrics.mean_squared_error(oof, train_y) ** 0.5,3))
rmse = round(10*log10(metrics.mean_squared_error(oof, train_y) ** 0.5),3)
print("CV score: {:<8.5f}".format(rmse))

#提交答案
test["pred"] = predictions

feature_importance.head(10)

fold 0


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[500]	training's rmse: 3.04772	valid_1's rmse: 3.11214
[1000]	training's rmse: 2.94572	valid_1's rmse: 3.03702
[1500]	training's rmse: 2.89006	valid_1's rmse: 3.00998
[2000]	training's rmse: 2.84595	valid_1's rmse: 2.99581
[2500]	training's rmse: 2.80925	valid_1's rmse: 2.98887
[3000]	training's rmse: 2.77623	valid_1's rmse: 2.98528
[3500]	training's rmse: 2.7456	valid_1's rmse: 2.98294
[4000]	training's rmse: 2.7175	valid_1's rmse: 2.98195
Early stopping, best iteration is:
[3920]	training's rmse: 2.72197	valid_1's rmse: 2.98181
fold 1
Training until validation scores don't improve for 100 rounds.
[500]	training's rmse: 3.04909	valid_1's rmse: 3.10388
[1000]	training's rmse: 2.94525	valid_1's rmse: 3.02908
[1500]	training's rmse: 2.88749	valid_1's rmse: 3.00021
[2000]	training's rmse: 2.84468	valid_1's rmse: 2.98791
[2500]	training's rmse: 2.80941	valid_1's rmse: 2.98188
Early stopping, best iteration is:
[2869]	training's

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,importance
Feature,
le_category,9286.2
hardvalue_sum,4594.6
hardvalue_last1,4462.6
score_std03,4314.4
score_std10,4000.2
score_std05,3746.2
score_std07,3552.2
score_mean03,3337.4
score_mean10,2909.0


In [17]:
sub =pd.read_csv(open(data_path+"submission_s2.csv",encoding='utf8'))
del sub["pred"]
sub = sub.merge(test[["student_id", "course", "exam_id","pred"]],how='left',on=["student_id", "course", "exam_id"])
sub.to_csv(out_path+"result_old_{}.csv".format(rmse),index=False)
sub.head()

,student_id,course,exam_id,pred
0,230748,course1,PcVABZEp,86.373045
1,186851,course1,PcVABZEp,82.087835
2,478370,course1,PcVABZEp,81.909700
3,692328,course1,PcVABZEp,81.812192
4,509128,course1,PcVABZEp,82.737889


# 模型2：期末考试模型

In [18]:
all_knowledge = pd.read_csv(open(data_path + 'all_knowledge.csv',encoding='utf8'))#2236
course1_exams = pd.read_csv(open(data_path + 'course1_exams.csv',encoding='utf8'))#85,369 考点依次向后，最后为综合性考试
course2_exams = pd.read_csv(open(data_path + 'course2_exams.csv',encoding='utf8')) #94,368
course3_exams = pd.read_csv(open(data_path + 'course3_exams.csv',encoding='utf8'))#70，263
course4_exams = pd.read_csv(open(data_path + 'course4_exams.csv',encoding='utf8'))#68,220
course5_exams = pd.read_csv(open(data_path + 'course5_exams.csv',encoding='utf8'))#90,336
course6_exams = pd.read_csv(open(data_path + 'course6_exams.csv',encoding='utf8'))#54,147
course7_exams = pd.read_csv(open(data_path + 'course7_exams.csv',encoding='utf8'))#78,317
course8_exams = pd.read_csv(open(data_path + 'course8_exams.csv',encoding='utf8'))#78,224
train = pd.read_csv(open(data_path + 'exam_score.csv',encoding='utf8')) #共8门课 65500,4/304500,4
student = pd.read_csv(open(data_path + 'student.csv',encoding='utf8'))#500,2
course = pd.read_csv(open(data_path + 'course.csv',encoding='utf8'))#8,2
test = pd.read_csv(open(data_path + 'submission_s2.csv',encoding='utf8'))#8000,5/4000,4 最后一次考试为最后半年的内容

In [19]:
#考试特征聚合
exams = pd.DataFrame({"exam_id":[],"level_1":[],0:[],"course":[]})
for i in range(1,9):
    s=0
    exec("""s = course{}_exams.set_index("exam_id").stack().to_frame().reset_index()""".format(i))
    exam_list = s["exam_id"].drop_duplicates().values
    s_temp = pd.DataFrame({"exam_id":exam_list,"exam_time":range(len(exam_list))})
    s = s.merge(s_temp,how='left',on="exam_id")
    s["course"] = "course{}".format(i)
    exams = exams.append(s)

exams = exams.rename(columns={0:"score"})

# 合并所有和考试相关的信息
exams = exams.merge(all_knowledge,how='left',left_on=["course","level_1"],right_on=["course","knowledge_point"])
exams = exams.merge(course,how='left',on='course')
exams["hardvalue"] = 0.01*exams["score"]*exams["complexity"] #难度值
exams["rank"] = exams.groupby("exam_id")["score"].rank(ascending=False,method='first')
exams["score"] = exams["score"].replace(0,np.nan)

#将所有考试信息归到一维
agg = {
    "score":["count"],#考了几个知识点
    "hardvalue":["sum","max"], #难度值、难度最高的知识点有多高
}
exams_group = exams.groupby(["exam_id","exam_time","course","course_class"],as_index=False).agg(agg) #617,2
exams_group.columns=["exam_id","exam_time","course","course_class","knowledge_cnt","hardvalue_sum","hardvalue_max"]
exams_group = exams_group.merge(exams.loc[exams["rank"]==1][["exam_id","section","category"]],how="left",on=["exam_id"])#本次考试的知识点所属领域
exams_group = exams_group.sort_values(["course","exam_time"])
# exam_all = exam_all.merge(exams_group,how='left',on=["exam_id","course"])
is_midterm={"course1":[5,17,27,37,48,58,68,78],
    "course2":[5,17,28,39,51,63,75,87],
    "course3":[4,12,21,30,40,48,56,65],
    "course4":[4,13,22,30,38,47,54,63],
    "course5":[4,15,27,38,49,59,70,83],
    "course6":[3,11,17,23,30,37,43,50],
    "course7":[4,14,24,34,44,54,63,72],
    "course8":[4,14,24,35,45,54,63,73]
}
is_final={"course1":[-1,11,22,31,43,53,63,72,84],
    "course2":[-1,11,22,33,45,57,69,80,93],
    "course3":[-1,8,16,25,35,44,52,60,69],
    "course4":[-1,8,17,26,33,43,50,58,67],
    "course5":[-1,9,21,33,43,54,64,76,89],
    "course6":[-1,7,14,20,26,33,40,46,53],
    "course7":[-1,8,19,29,39,49,58,67,77],
    "course8":[-1,9,19,29,40,49,58,68,77]
}
exams_group["is_midterm"] = 0 #是否期中
exams_group["is_final"] = 0 #是否期末
exams_group["term"] = 0 #学期数
for key in is_midterm.keys():
    exams_group.loc[(exams_group["course"]==key)&(exams_group["exam_time"].isin(is_midterm[key])),"is_midterm"]=1
    exams_group.loc[(exams_group["course"] == key) & (exams_group["exam_time"].isin(is_final[key])), "is_final"] = 1
    term = exams_group.loc[exams_group["course"]==key,"exam_time"]
    exams_group.loc[exams_group["course"] == key, "term"] = pd.cut(term,is_final[key],labels=False)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [20]:
#学生维度
train["is_train"]=1
test["is_train"]=0
test.rename(columns = {'pred':'score'},inplace=True)
data_all = train.append(test)
data_all = data_all.merge(student,how='left',on='student_id')
data_all = data_all.merge(exams_group,how='left',on=["exam_id","course"])
data_all["effotvalue"] = data_all["score"]/data_all["hardvalue_sum"]
bin_labels = [0,1,2,3,4,5,6,7]
data_all["hardvalue_rank"]=pd.cut(data_all["hardvalue_sum"], 8, right=True, labels=bin_labels, retbins=False, precision=3, include_lowest=False)
#当前章节第几次考试,测试集中3/16次考试更换了知识点,category_rank为nan则说明为阶段性考试
data_tmp = data_all.sort_values(by=["student_id","course","section","knowledge_cnt"])
data_tmp = data_tmp.drop_duplicates(["student_id","course","section"])
data_tmp["category_rank"] = data_tmp.groupby(["student_id","course","category"])["exam_time"].rank(method="first")
data_all = data_all.merge(data_tmp[["student_id","course","exam_time","category_rank"]],how='left',on=["student_id","course","exam_time"])

#每学期每门考试的统计
data_course_group = data_all.loc[(data_all.is_final==0)&(data_all.score!=0)]
agg = {
    "score":["mean","median","std","max","min",skew,kurtosis],
    "effotvalue":["mean","median","std","max","min",skew,kurtosis],
}
data_course_group = data_course_group.groupby(["term","course"],as_index=False).agg(agg)
data_course_group.columns = ["term","course"]+["course_group_"+i[0]+"_"+i[1] for i in data_course_group.columns[2:]]

#对所有考试展开
every_exam = data_all.loc[(data_all.is_final==0)&(data_all.is_midterm==0)]
every_exam["exam_rank"] = every_exam.groupby(["term","course","student_id"])["exam_time"].rank(method='first')

#每个章节的分数
exams_final = exams.loc[exams.exam_id.isin(set(data_all[data_all.is_final==1].exam_id))]
exams_final = exams_final.dropna(axis=0,how='any').rename(columns={"exam_time":"exam_time_final"})
exams_final_tmp = exams_final.groupby(["course","section"],as_index=False)["score"].sum()
exams_final_tmp = exams_final_tmp.rename(columns={"score":"section_score"})
# exams_final_tmp[:20]
every_exam = every_exam.merge(exams_final_tmp,how='left',on=["course","section"])
every_exam[every_exam.student_id==230748]
#data_all为全部数据
for i in range(1,12):
    print(i)
    every_exam_tmp = every_exam.loc[every_exam.exam_rank==i][["term","course","student_id","hardvalue_sum","score","section_score"]]
    every_exam_tmp["score"] = every_exam_tmp["score"].replace(0,np.nan)
    every_exam_tmp.columns = ["term","course","student_id","last_hard{}".format(i),"last_exam{}".format(i),"section_score{}".format(i)]
    data_all = data_all.merge(every_exam_tmp,how='left',on=["term","course","student_id"])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1
2
3
4
5
6
7
8
9
10
11


In [21]:
#提取所有期末考试

#对0分学生处理
print("score==0:",data_all.loc[data_all.score==0].shape[0])
data_all["is_zero"] = data_all.score.apply(lambda x: 1 if x ==0 else 0)
data_all_zero = data_all.loc[data_all.is_final==0] #去除期末考试
#计算一个学生所有0分情况
data_all_zero = data_all_zero.groupby("student_id",as_index=False)["is_zero"].sum()
data_all_zero = data_all_zero.rename(columns={"is_zero":"his_zero_cnt"})
data_all = data_all.merge(data_all_zero,how='left',on="student_id")

print("data_all shape:",data_all.shape)
#每次期末考试大概有8个0分学生
data_all = data_all.loc[(data_all.score!=0)|(data_all.is_final==1)]#选择所有期末考试及非期末中非0考试
print("data_all shape:",data_all.shape)
data_final = data_all.loc[data_all.is_final==1]

######统计特征1：对该学期所有考试进行统计（包括期中）###################################################################
data_group = data_all.loc[data_all.is_final==0]
from scipy.stats import skew,kurtosis

#针对一个学生一门课程一学期考试做统计
agg = {
    "score":["mean","median","std","max","min",skew,kurtosis],
    "effotvalue":["mean","median","std","max","min",skew,kurtosis],
    "section":["count"]
}
data_group = data_group.groupby(["student_id","course","term"],as_index=False).agg(agg)
data_group.columns = ["student_id","course","term"]+[i[0]+"_"+i[1] for i in data_group.columns[3:]]
data_final = data_final.merge(data_group,how='left',on=["student_id","course","term"])

#上一次期中考试及难度
data_midterm = data_all.loc[data_all.is_midterm==1][["student_id","course","term","score","hardvalue_sum"]]
data_midterm = data_midterm.rename(columns={"score":"last_exam_midterm","hardvalue_sum":"last_hard_midterm"})
print("期中考试为0的数量",data_midterm[data_midterm.last_exam_midterm==0].shape)
data_final = data_final.merge(data_midterm,how='left',on=["student_id","course","term"])

######统计特征2：针对期末的每个知识点找到对应的单科考试成绩并加权#############################################
#针对考试的知识点做统计 筛选非期中期末的考试
data_group2 = data_all.loc[(data_all.is_final==0)&(data_all.is_midterm==0)][["student_id","course","term","score","hardvalue_sum","section","exam_time"]]
data_group2 = data_group2.sort_values(["student_id","course","term","exam_time"])
data_group2 = data_group2.rename(columns={"score":"his_score","hardvalue_sum":"his_hard"})
#提取期末考试的知识点 score为当前知识点score
exams_final = exams.loc[exams.exam_id.isin(set(data_final.exam_id))]
exams_final = exams_final.dropna(axis=0,how='any').rename(columns={"exam_time":"exam_time_final"})

exams_final_tmp = exams_final.groupby(["exam_id","section"],as_index=False)["score"].sum()
exams_final_tmp.columns = ["exam_id","section","score_sum"]
exams_final = exams_final.merge(exams_final_tmp,how='left',on=["exam_id","section"])
exams_final["score_hard"] = (exams_final["score"]/exams_final["score_sum"])*exams_final["complexity"]
#当前章节分数及难度
exams_final_group = exams_final.groupby(["exam_id","course","exam_time_final","section"],as_index=False).agg({"score_sum":"mean","score_hard":"sum"})
exams_final_group = exams_final_group.rename(columns={"score_sum":"score"})

exams_final = exams_final_group.merge(data_group2,how="left",on=["course","section"])
exams_final = exams_final.sort_values(["student_id","course","term","exam_time"])

#有学生章节考试为0分，故应考虑该情况
exams_final_tmp2 = exams_final.groupby(["student_id","exam_id"],as_index=False)["score"].sum()
exams_final_tmp2 = exams_final_tmp2.rename(columns={"score":"score_sum"})
exams_final = exams_final.merge(exams_final_tmp2,how = 'left',on=["student_id","exam_id"])

#score为期末考试每个章节分数，his_score为该学生历史章节考试成绩，his_hard为历史章节考试难度，score_hard为期末考试各章节难度
# mean 6.76/1 6.67/2 6.63 7.34/3 6.60/4 6.58/5 6.57###
# 3 4.305/4 4.253/5 4.227/6 4.226
exams_final["pred_score"] = (exams_final["score"]/exams_final["score_sum"])*(exams_final["his_score"]+(exams_final["his_hard"]-exams_final["score_hard"])*4)
exams_final_group2 = exams_final.groupby(["exam_id","course","student_id","term"],as_index=False)["pred_score"].sum()

#合并
data_final = data_final.merge(data_course_group,how='left',on=["term","course"])
data_final = data_final.merge(exams_final_group2,how='left',on=["exam_id","student_id","course","term"])
data_final = data_final.sort_values(["student_id","course","term"])
#上一次期末考试及难度
data_final["last_exam_final"] = data_final.groupby(["course","student_id"])["score"].shift(1)
data_final["last_hard_final"] = data_final.groupby(["course","student_id"])["hardvalue_sum"].shift(1)
data_final["last_exam_final2"] = data_final.groupby(["course","student_id"])["score"].shift(2)
data_final["last_hard_final2"] = data_final.groupby(["course","student_id"])["hardvalue_sum"].shift(2)
data_final["last_exam_final3"] = data_final.groupby(["course","student_id"])["score"].shift(3)
data_final["last_hard_final3"] = data_final.groupby(["course","student_id"])["hardvalue_sum"].shift(3)
data_final["last_exam_final4"] = data_final.groupby(["course","student_id"])["score"].shift(4)
data_final["last_hard_final4"] = data_final.groupby(["course","student_id"])["hardvalue_sum"].shift(4)

# 比例特征
#章节考试之间的比例
data_final["ratio_last_exam12"] = data_final["last_exam1"]/data_final["last_exam2"]
data_final["ratio_last_exam23"] = data_final["last_exam2"]/data_final["last_exam3"]
data_final["ratio_last_exam34"] = data_final["last_exam3"]/data_final["last_exam4"]
data_final["ratio_last_exam45"] = data_final["last_exam4"]/data_final["last_exam5"]
data_final["ratio_last_exam56"] = data_final["last_exam5"]/data_final["last_exam6"]
data_final["ratio_last_exam67"] = data_final["last_exam6"]/data_final["last_exam7"]
data_final["ratio_last_exam78"] = data_final["last_exam7"]/data_final["last_exam8"]
data_final["ratio_last_exam89"] = data_final["last_exam8"]/data_final["last_exam9"]
data_final["ratio_last_exam910"] = data_final["last_exam9"]/data_final["last_exam10"]
data_final["ratio_last_exam1011"] = data_final["last_exam10"]/data_final["last_exam11"]

data_final = data_final.sort_values(["student_id","term","course","exam_time"])

score==0: 601
data_all shape: (308500, 54)
data_all shape: (307955, 54)
期中考试为0的数量 (0, 5)


In [22]:
from gensim.models import Word2Vec
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

def embedding(series, size=8, ids=None, sp_weights=None, combiner=None):
    """
    :param series: 要进行embedding的列,每行数据格式为["dog","cat","pig"]或者"dog cat pig"
    :param ids：所有word的排序["cat","dog","pig"]
    :param size: w2v的维度
    :param sp_weights: 各类别的权重，是否加权，必须和ids具有相同的维度
    :param combiner: 默认‘mean’，还可以选择 "sqrtn" "sum"
    :return:word_vec w2v结果
    :return:result_embedding[0] embedding结果
    """
    type_s = type(series[0]).__name__
    if type_s == 'list':
        series_list = series.copy()
        series_str = series.apply(lambda x: " ".join(x))
    elif type_s == 'str':
        series_list = series.apply(lambda x: x.split(" "))
        series_str = series.copy()
    if ids == None:
        ids = sorted(list(set(series_list.sum())))

    word_list = []
    for word in series_list:
        word_list.append(word)
    #参数根据实际情况修改
    model = Word2Vec(word_list, size=size, window=3,
                     min_count=1, workers=1,
                     iter=20, seed=2019)
    print("行为种类", len(ids))

    # 构造vec ids对应的vec，维度为 ids个数*n
    word_vec = []
    for id in ids:
        word_vec.append(list(model[id]))
    word_vec = np.array(word_vec).astype(np.float32)
    print("w2v维度", len(word_vec), len(word_vec[0]))
    params = tf.Variable(word_vec)

    # word count编码 若不加a则算法无法识别'0'~'9'
    cv = CountVectorizer(min_df=1, max_df=999999)
    alert_cv = cv.fit_transform(series_str)  # 元素位置 及值
    name = cv.vocabulary_
    name2 = sorted(name.items(), key=lambda x: x[1], reverse=False)
    name_id = [kk[0] for kk in name2]

    print("w2v和cv顺序是否一致", ids == name_id)

    #获取word count矩阵的行列位置，及对应的值
    cv_row =list(alert_cv.tocoo().row.reshape(-1)) #cv的行
    cv_col = list(alert_cv.tocoo().col.reshape(-1)) #cv的列
    col_cnt = alert_cv.data.tolist()#col出现的次数
    cv_df = pd.DataFrame({"cv_row":cv_row,"cv_col":cv_col,"col_cnt":col_cnt})

    # 按照出现的次数展开
    cv_row_new = []
    cv_col_new = []
    for ind in range(cv_df.shape[0]):
        cv_row_new.extend([cv_df["cv_row"][ind]] * cv_df["col_cnt"][ind])
        cv_col_new.extend([cv_df["cv_col"][ind]] * cv_df["col_cnt"][ind])
    cv_df_new = pd.DataFrame({"cv_row": cv_row_new, "cv_col": cv_col_new})
    cv_df_new["rank"] = cv_df_new.groupby("cv_row")["cv_col"].rank(ascending=True, method="first")
    cv_df_new["rank"] = (cv_df_new["rank"] - 1).astype(int)

    # 组合indices和values
    indices = []
    values = []
    for ind in range(cv_df_new.shape[0]):
        indices.append([cv_df_new["cv_row"][ind], cv_df_new["rank"][ind]])
        values.append(cv_df_new["cv_col"][ind])

    # 构造embedding输入
    tags = tf.SparseTensor(indices=indices, values=values,
                           dense_shape=(cv_df_new["cv_row"].max() + 1, cv_df_new["rank"].max() + 1))
    # a = tf.SparseTensor(indices=[[0, 0], [1, 2], [1, 3]], values=[1, 2, 3], dense_shape=[2, 4])
    embedding_tags = tf.nn.embedding_lookup_sparse(params, sp_ids=tags, sp_weights=sp_weights, combiner=combiner)

    with tf.Session() as s:
        s.run([tf.global_variables_initializer(), tf.tables_initializer()])
        result_embedding = s.run([embedding_tags])
    print("embedding完成,维度：{}".format(result_embedding[0].shape))
    return word_vec,result_embedding[0]

In [23]:
def emb_trans(df):
    tmp = []
    for i in range(df["section_count"]-1):
        tt=df['last_exam{}'.format(i+1)]
        try:
            tt = 'a'+str(int(tt))
        except:
            tt='a'+str(tt)
        tmp.append(tt)   
    return tmp
data_final["emb_exam_tmp"] = data_final.apply(emb_trans,axis=1)

exam_emb = embedding(data_final["emb_exam_tmp"],size=2)[1]
exam_emb = pd.DataFrame(exam_emb,columns=["emb_exam0","emb_exam1",
                                            # "emb_exam2","emb_exam3"
                                            ])
data_final = pd.concat([data_final,exam_emb],axis=1)
del data_final["emb_exam_tmp"]

行为种类 51
w2v维度 51 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


w2v和cv顺序是否一致 True
embedding完成,维度：(32000, 2)


In [32]:
data_final_temp = data_final.copy()
data_final_temp = data_final_temp.loc[data_final_temp.score!=0] #包含0训练 6.55 7.39/ 不包含4.04 7.316
le = LabelEncoder()
data_final_temp["le_course"] = le.fit_transform(data_final_temp["course"])
le = LabelEncoder()
data_final_temp["le_course_class"] = le.fit_transform(data_final_temp["course_class"])
le = LabelEncoder()
data_final_temp["le_category"] = le.fit_transform(data_final_temp["category"])

y = "score"

features = ["gender","exam_time","knowledge_cnt"]\
           +["hardvalue_sum","hardvalue_max","hardvalue_rank"] \
           + [i for i in data_final_temp.columns if i[:6] == "score_"] \
           +["le_course","le_course_class"] \
           + [i for i in data_final_temp.columns if i[:11]=="effotvalue_"] \
           + [i for i in data_final_temp.columns if i[:5]=="last_"] \
           + ["section_count","pred_score","his_zero_cnt"] \
          + [i for i in data_final_temp.columns if i[:9]=="section_s"]\
        + [i for i in data_final_temp.columns if "course_group" in i]\
        + [i for i in data_final_temp.columns if i[:6]=="ratio_"]\

print(features)
print(len(features))
categorical_feats = ["le_course","le_course_class","gender"]
train = data_final_temp[data_final_temp.is_train==1]
test = data_final_temp[data_final_temp.is_train==0]
print("train shape:",train.shape)
print("test shape:",test.shape)
train[features+[y]].head()

['gender', 'exam_time', 'knowledge_cnt', 'hardvalue_sum', 'hardvalue_max', 'hardvalue_rank', 'score_mean', 'score_median', 'score_std', 'score_max', 'score_min', 'score_skew', 'score_kurtosis', 'le_course', 'le_course_class', 'effotvalue_mean', 'effotvalue_median', 'effotvalue_std', 'effotvalue_max', 'effotvalue_min', 'effotvalue_skew', 'effotvalue_kurtosis', 'last_hard1', 'last_exam1', 'last_hard2', 'last_exam2', 'last_hard3', 'last_exam3', 'last_hard4', 'last_exam4', 'last_hard5', 'last_exam5', 'last_hard6', 'last_exam6', 'last_hard7', 'last_exam7', 'last_hard8', 'last_exam8', 'last_hard9', 'last_exam9', 'last_hard10', 'last_exam10', 'last_hard11', 'last_exam11', 'last_exam_midterm', 'last_hard_midterm', 'last_exam_final', 'last_hard_final', 'last_exam_final2', 'last_hard_final2', 'last_exam_final3', 'last_hard_final3', 'last_exam_final4', 'last_hard_final4', 'section_count', 'pred_score', 'his_zero_cnt', 'section_score1', 'section_score2', 'section_score3', 'section_score4', 'sectio

,gender,exam_time,knowledge_cnt,hardvalue_sum,hardvalue_max,hardvalue_rank,score_mean,score_median,score_std,score_max,score_min,score_skew,score_kurtosis,le_course,le_course_class,effotvalue_mean,effotvalue_median,effotvalue_std,effotvalue_max,effotvalue_min,effotvalue_skew,effotvalue_kurtosis,last_hard1,last_exam1,last_hard2,last_exam2,last_hard3,last_exam3,last_hard4,last_exam4,last_hard5,last_exam5,last_hard6,last_exam6,last_hard7,last_exam7,last_hard8,last_exam8,last_hard9,last_exam9,last_hard10,last_exam10,last_hard11,last_exam11,last_exam_midterm,last_hard_midterm,last_exam_final,last_hard_final,last_exam_final2,last_hard_final2,last_exam_final3,last_hard_final3,last_exam_final4,last_hard_final4,section_count,pred_score,his_zero_cnt,section_score1,section_score2,section_score3,section_score4,section_score5,section_score6,section_score7,section_score8,section_score9,section_score10,section_score11,course_group_score_mean,course_group_score_median,course_group_score_std,course_group_score_max,course_group_score_min,course_group_score_skew,course_group_score_kurtosis,course_group_effotvalue_mean,course_group_effotvalue_median,course_group_effotvalue_std,course_group_effotvalue_max,course_group_effotvalue_min,course_group_effotvalue_skew,course_group_effotvalue_kurtosis,ratio_last_exam12,ratio_last_exam23,ratio_last_exam34,ratio_last_exam45,ratio_last_exam56,ratio_last_exam67,ratio_last_exam78,ratio_last_exam89,ratio_last_exam910,ratio_last_exam1011,score
0,0,11.0,18,2.73,0.33,5,72.454545,71.0,4.227615,80.0,67.0,0.400107,-0.863984,0,0,34.465692,27.131783,12.094943,52.631579,22.1875,0.525633,-1.579696,2.37,71.0,3.2,71.0,2.72,70.0,2.83,73.0,1.59,76.0,2.77,67.0,1.52,80.0,1.59,78.0,1.53,74.0,2.55,67.0,NaN,NaN,70.0,2.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,71.9808,1,15.0,7.0,6.0,21.0,7.0,7.0,5.0,29.0,3.0,NaN,NaN,82.331088,82.0,8.807722,100.0,57.0,-0.061393,-0.577691,39.139228,32.941176,13.403911,65.789474,17.8125,0.534961,-1.262051,1.000000,1.014286,0.958904,0.960526,1.134328,0.837500,1.025641,1.054054,1.104478,NaN,70.0
1,0,11.0,18,2.73,0.33,5,81.090909,79.0,9.278519,98.0,67.0,0.391559,-0.752433,0,0,38.983752,31.645570,15.405873,64.473684,20.9375,0.568460,-1.303589,2.37,75.0,3.2,67.0,2.72,76.0,2.83,73.0,1.59,79.0,2.77,84.0,1.52,98.0,1.59,93.0,1.53,88.0,2.55,84.0,NaN,NaN,75.0,2.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,79.7308,1,15.0,7.0,6.0,21.0,7.0,7.0,5.0,29.0,3.0,NaN,NaN,82.331088,82.0,8.807722,100.0,57.0,-0.061393,-0.577691,39.139228,32.941176,13.403911,65.789474,17.8125,0.534961,-1.262051,1.119403,0.881579,1.041096,0.924051,0.940476,0.857143,1.053763,1.056818,1.047619,NaN,75.0
2,0,11.0,18,2.73,0.33,5,81.545455,84.0,5.820028,88.0,72.0,-0.458629,-1.321737,0,0,38.434065,34.108527,12.201636,55.921053,24.6875,0.413199,-1.523667,2.37,86.0,3.2,79.0,2.72,87.0,2.83,80.0,1.59,87.0,2.77,74.0,1.52,85.0,1.59,84.0,1.53,75.0,2.55,72.0,NaN,NaN,88.0,2.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,80.5608,0,15.0,7.0,6.0,21.0,7.0,7.0,5.0,29.0,3.0,NaN,NaN,82.331088,82.0,8.807722,100.0,57.0,-0.061393,-0.577691,39.139228,32.941176,13.403911,65.789474,17.8125,0.534961,-1.262051,1.088608,0.908046,1.087500,0.919540,1.175676,0.870588,1.011905,1.120000,1.041667,NaN,75.0
3,1,11.0,18,2.73,0.33,5,83.818182,82.0,5.231026,95.0,78.0,1.158660,0.224593,0,0,39.894798,31.782946,14.099267,62.500000,24.3750,0.553162,-1.442611,2.37,81.0,3.2,78.0,2.72,82.0,2.83,83.0,1.59,84.0,2.77,79.0,1.52,95.0,1.59,92.0,1.53,85.0,2.55,81.0,NaN,NaN,82.0,2.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,83.3308,2,15.0,7.0,6.0,21.0,7.0,7.0,5.0,29.0,3.0,NaN,NaN,82.331088,82.0,8.807722,100.0,57.0,-0.061393,-0.577691,39.139228,32.941176,13.403911,65.789474,17.8125,0.534961,-1.262051,1.038462,0.951220,0.987952,0.988095,1.063291,0.831579,1.032609,1.082353,1.049383,NaN,83.0
4,0,11.0,18,2.73,0.33,5,77.818182,77.0,5.153992,87.0,71.0,0.564558,-0.629030,0,0,37.087428,30.980392,13.230685,57.236842,22.5000,0.506874,-1.510406,2.37,75.0,3.2,72.0,2.72,74.0,2.83,76.0,1.59,79.0,2.77,71.0,1.52,87.0,1.59,86.0,1.53,80.0,2.55

In [33]:
folds = KFold(n_splits=5, shuffle=True, random_state=2333)

oof = np.zeros(len(train))
predictions = np.zeros(len(test))
train_x = train[features]
test_x = test[features]
train_y = train[y]
stack_metrix = np.array([len(train),5])
param = {'objective': 'regression',
         'num_leaves': 24, #2**5 32 3.983 28 3.980 29 3.979
         'min_data_in_leaf': 25,#25
         'max_depth': 5,  #5
         'learning_rate': 0.02, #
         'lambda_l1': 0.13,#0.13
         "boosting": "gbdt",
         "feature_fraction": 0.7,#0.7
         'bagging_freq': 8,#8
         "bagging_fraction": 0.9, #0.9
         "metric": 'rmse',
         "verbosity": -1,
         "random_state": 2333,
         "num_threads" : 50}
# lgb
model = "lgb"
feature_importance_df = pd.DataFrame()
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x.values, train_y.values)):
    print("fold {}".format(fold_))
    print("features:",len(features))

    trn_data = lgb.Dataset(train_x.iloc[trn_idx],
                          label=train_y.iloc[trn_idx],
                          categorical_feature=categorical_feats
                          )
    val_data = lgb.Dataset(train_x.iloc[val_idx],
                          label=train_y.iloc[val_idx],
                          categorical_feature=categorical_feats
                          )

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round
                    , valid_sets=[trn_data, val_data]
                    , verbose_eval=300
                    ,early_stopping_rounds=100
                    # ,feval=custom_mertic
                    )

    oof[val_idx] = clf.predict(train_x.iloc[val_idx], num_iteration=clf.best_iteration)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    predictions += clf.predict(test_x, num_iteration=clf.best_iteration) / folds.n_splits
    
feature_importance = feature_importance_df[["Feature", "importance"]].groupby("Feature").mean().sort_values(by="importance",
ascending=False)

rmse = round(10*log10(mean_squared_error(train_y, oof)**0.5),3)

print("rmse",mean_squared_error(train_y, oof)**0.5)

print("cv score: {:<8.5f}".format(rmse))

test["pred"] = predictions

feature_importance.head()

fold 0
features: 92


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[300]	training's rmse: 2.40311	valid_1's rmse: 2.52885
[600]	training's rmse: 2.30549	valid_1's rmse: 2.51094
[900]	training's rmse: 2.23375	valid_1's rmse: 2.50658
Early stopping, best iteration is:
[953]	training's rmse: 2.22103	valid_1's rmse: 2.50545
fold 1
features: 92
Training until validation scores don't improve for 100 rounds.
[300]	training's rmse: 2.40714	valid_1's rmse: 2.5132
[600]	training's rmse: 2.31216	valid_1's rmse: 2.49447
[900]	training's rmse: 2.24198	valid_1's rmse: 2.49135
Early stopping, best iteration is:
[936]	training's rmse: 2.23384	valid_1's rmse: 2.49041
fold 2
features: 92
Training until validation scores don't improve for 100 rounds.
[300]	training's rmse: 2.41132	valid_1's rmse: 2.49743
[600]	training's rmse: 2.3176	valid_1's rmse: 2.48265
Early stopping, best iteration is:
[714]	training's rmse: 2.28882	valid_1's rmse: 2.48085
fold 3
features: 92
Training until validation scores don't impr

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,importance
Feature,
pred_score,1333.8
ratio_last_exam45,609.2
ratio_last_exam34,586.6
ratio_last_exam12,586.0
last_exam1,540.6


In [34]:
sub =pd.read_csv(open(data_path+"submission_s2.csv",encoding='utf8'))
print(sub.head())
del sub["pred"]
sub = sub.merge(test[["student_id", "course", "exam_id","pred"]],how='left',on=["student_id", "course", "exam_id"])
name = "result_{}_{}.csv".format(model,rmse)
print(name)
sub.to_csv(out_path+name,index=False)
sub.head()

   student_id   course   exam_id  pred
0      230748  course1  PcVABZEp   NaN
1      186851  course1  PcVABZEp   NaN
2      478370  course1  PcVABZEp   NaN
3      692328  course1  PcVABZEp   NaN
4      509128  course1  PcVABZEp   NaN
result_lgb_3.968.csv


,student_id,course,exam_id,pred
0,230748,course1,PcVABZEp,85.614003
1,186851,course1,PcVABZEp,79.305076
2,478370,course1,PcVABZEp,82.770960
3,692328,course1,PcVABZEp,79.647450
4,509128,course1,PcVABZEp,79.999485


# 模型融合

In [35]:
model1 = pd.read_csv(out_path+"result_old_4.732.csv") #
model2 = pd.read_csv(out_path+name) #7.302
model_out = model1.copy()
model_out["pred"] = 0.35*model1["pred"]+0.65*model2["pred"]
model_out.to_csv(out_path+"stack_3565.csv",index=False)
model_out.head()

,student_id,course,exam_id,pred
0,230748,course1,PcVABZEp,85.917620
1,186851,course1,PcVABZEp,80.418180
2,478370,course1,PcVABZEp,82.426456
3,692328,course1,PcVABZEp,80.513347
4,509128,course1,PcVABZEp,81.094846


In [36]:
!wget -O tianyi_submit https://cdn.kesci.com/submit_tool/v3/tianyi_submit&&chmod +x tianyi_submit

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-09-05 18:08:38--  https://cdn.kesci.com/submit_tool/v3/tianyi_submit
Resolving cdn.kesci.com (cdn.kesci.com)... 42.120.107.18, 42.120.107.17
Connecting to cdn.kesci.com (cdn.kesci.com)|42.120.107.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6709558 (6.4M) [application/x-executable]
Saving to: ‘tianyi_submit’

tianyi_submit       100%[===================>]   6.40M  --.-KB/s    in 0.1s    

2019-09-05 18:08:38 (59.6 MB/s) - ‘tianyi_submit’ saved [6709558/6709558]



In [37]:
!./tianyi_submit -file /home/kesci/work/out/stack_46.csv -token 9f743b5e40d4520c

Tianyi Submit Tool 3.2.1

> 已验证Token
> 提交文件 /home/kesci/work/out/stack_46.csv (163.44 KiB)
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
